In [ ]:
import sys
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean as cmo
import cartopy.crs as ccrs

sys.path.append("..")

from plotfunctions import *

np.seterr(all='ignore')
import warnings
warnings.filterwarnings("ignore")

%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
mpl.rcParams['figure.subplot.bottom'] = .02
mpl.rcParams['figure.subplot.top']    = .94
mpl.rcParams['figure.subplot.left']   = .01
mpl.rcParams['figure.subplot.right']  = .88
mpl.rcParams['figure.subplot.wspace'] = .02
mpl.rcParams['figure.subplot.hspace'] = .0

In [ ]:
"""Boundaries Adusumilli"""
X1,X2 = 2120,2480
Y1,Y2 = {},{}
Y1['CrossDots'] = 3360
Y2['CrossDots'] = 3680
Y1['Thwaites_e'] = 3680
Y2['Thwaites_e'] = 4000
Y1['PineIsland']  = 4000
Y2['PineIsland']  = 4240

"""Boundaries MITgcm"""
y1,y2 = -75.45,-74.05
x1,x2 = {},{}
x1['CrossDots'] = 245.3
x2['CrossDots'] = 251
x1['Thwaites_e'] = 251
x2['Thwaites_e'] = 257
x1['PineIsland']  = 257
x2['PineIsland']  = 262

"""Locations for text"""
plon,plat = {},{}
plon['CrossDots'] = 250.5
plat['CrossDots'] = -74.7
plon['Thwaites_e'] = 253.5
plat['Thwaites_e'] = -74.7
plon['PineIsland'] = 257.9
plat['PineIsland'] = -75.1

In [ ]:
#Parameters MITgcm
timep= slice("1989-1-1","2018-12-31")


In [ ]:
fig = plt.figure(figsize=(8,7))
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=-13)

ax1 = fig.add_subplot(131,projection=proj)
makebackground(ax1)
ax1.set_title('Observations')
ax2 = fig.add_subplot(132,projection=proj)
makebackground(ax2)
ax2.set_title('3D Ocean model')
ax3 = fig.add_subplot(133,projection=proj)
makebackground(ax3)
ax3.set_title('2D Layer model')

for geom in ['PineIsland','Thwaites_e','CrossDots']:
    """Adusumilli"""

    ds = xr.open_dataset('../../data/bb0448974g_3_1.h5')
    ds = ds.isel(phony_dim_0=slice(Y1[geom],Y2[geom]),phony_dim_1=slice(X1,X2),phony_dim_2=0)
    ds = add_lonlat(ds)

    melt = xr.where(np.isnan(ds.w_b),0,ds.w_b)
    melt += xr.where(np.isnan(ds.w_b_interp),0,ds.w_b_interp)
    melt = xr.where(melt == 0,np.nan,melt)
    
    ax1.text(plon[geom],plat[geom],f'{np.nanmean(melt):.1f} ({np.nanstd(melt):.1f})',transform=ccrs.PlateCarree(),c='w',ha='right')
    IM = plotnormmelt(ax1,ds.lon,ds.lat,melt)

    """MITgcm output"""

    ds = xr.open_dataset('../../data/paulholland/PAS_851/state2D.nc')
    ds = ds.sel(LONGITUDE=slice(x1[geom],x2[geom]),LATITUDE=slice(y1,y2),TIME=timep)
    ds = ds.mean(dim='TIME')
    lon = ds.LONGITUDE
    lat = ds.LATITUDE-.05
    
    melt = -ds.SHIfwFlx * 365.25*86400/920
    melt = xr.where(melt==0,np.nan,melt)
    
    ax2.text(plon[geom],plat[geom],f'{np.nanmean(melt):.1f} ({np.nanstd(melt):.1f})',transform=ccrs.PlateCarree(),c='w',ha='right')
    IM = plotnormmelt(ax2,lon,lat,melt)

    """Layer"""
    ds = xr.open_dataset(f'../../results/Layer_{geom}_holland.nc')
    ds = add_lonlat(ds)
    
    melt = np.where(ds.mask==3,ds.melt,np.nan)

    ax3.text(plon[geom],plat[geom],f'{np.nanmean(melt):.1f} ({np.nanstd(melt):.1f})',transform=ccrs.PlateCarree(),c='w',ha='right')
    IM = plotnormmelt(ax3,ds.lon,ds.lat,melt)

"""Colorbar"""
ax = fig.add_subplot(133)
ax.set_visible(False)
axins = inset_axes(ax,width="5%",height="100%",loc='lower left',bbox_to_anchor=(1.05, 0., 1, 1),bbox_transform=ax.transAxes,borderpad=0)
cbar = plt.colorbar(IM, cax=axins,extend='both')

cbar.set_label('Spatial standardised melt rate anomaly []', labelpad=0)

"""Save figure"""
plt.savefig(f"../../figures/Validation_standardized.png",dpi=300)
plt.show()